In [3]:
import numpy as np

from causalchamber.datasets import Dataset
from crc.eval import compute_MCC


Fetching list of available datasets from https://causalchamber.s3.eu-central-1.amazonaws.com/downloadables/directory.yaml ... done.
If you use our datasets or models for your work please consider citing:

@article{gamella2024chamber,
  title={The Causal Chambers: Real Physical Systems as a Testbed for AI Methodology},
  author={Gamella, Juan L. and B"uhlmann, Peter and Peters, Jonas},
  journal={arXiv preprint arXiv:2404.11341},
  year={2024}
}



In [10]:
ica_dataset1 = Dataset('lt_walks_v1', root='../data/chamber_downloads', download=True)

ica_experiment1 = ica_dataset1.get_experiment(name='color_mix')
ica_df1 = ica_experiment1.as_pandas_dataframe()

Dataset lt_walks_v1 found in "../data/chamber_downloads/lt_walks_v1".


In [11]:
ica_df1

,timestamp,config,counter,flag,intervention,red,green,blue,osr_c,v_c,...,diode_vis_3,t_ir_1,t_vis_1,t_ir_2,t_vis_2,t_ir_3,t_vis_3,camera,v_board,v_reg
0,20969.269894,standard,0.0,0.0,1.0,152.0,255.0,25.0,1.0,5.0,...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,1023.0,699.0
1,20969.396700,standard,1.0,0.0,1.0,153.0,255.0,25.0,1.0,5.0,...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,1023.0,862.0
2,20969.523702,standard,2.0,0.0,1.0,153.0,255.0,25.0,1.0,5.0,...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,1023.0,931.0
3,20969.650758,standard,3.0,0.0,1.0,153.0,255.0,26.0,1.0,5.0,...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,1023.0,963.0
4,20969.777684,standard,4.0,0.0,1.0,153.0,255.0,26.0,1.0,5.0,...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,1023.0,978.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,22219.558586,standard,9995.0,0.0,1.0,91.0,0.0,254.0,1.0,5.0,...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,1023.0,990.0
9996,22219.682434,standard,9996.0,0.0,1.0,91.0,0.0,254.0,1.0,5.0,...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,1023.0,991.0
9997,22219.806029,standard,9997.0,0.0,1.0,91.0,0.0,254.0,1.0,5.0,...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,1023.0,990.0
9998,22219.931040,standard,9998.0,0.0,1.0,90.0,0.0,254.0,1.0,5.0,...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,1023.0,990.0
